In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import json
import os

import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
from geoalchemy2 import Geometry, WKTElement

# Data Cleaning and Uploading Data to pgAdmin

In the section below, we are cleaning all our files including:
1.  Neighbourhoods.csv
2.  BusinessStats.csv
3.  StatisticalAreas.csv
4.  RFSNSW_BFPL.shp
5.  SA2_2016_AUST.shp
6.  RFSStation_EPSG4326.json (additional dataset)

Methods for cleaning data includes:

- checkNull(dataset, beenCleaned): check if there's a null value
- removeNull(dataset): drop the null value in the dataset

** For each dataset, we also check the type of different columns to see if they are in their correct type. If they're not, then we convert them. For instance, all population values are intially string, but we convert them into numeric values.


In [2]:
# Function use to check if null values exist in the dataset. If there's null value, we'll drop the row to increase 
# accuracy in our calculations.

def checkNull( dataset ):
    
    beenCleaned = False    
    
    for num in list(dataset.isnull().sum()):

        if num != 0:
            print("THIS DATASET HAS SOME NA VALUES.")
            print("Null value found. Coverted null values to ...")
            dataset = dataset.dropna() #found null, drop the row
            beenCleaned = True
            
    if beenCleaned:
        print("\nNull values have been removed. You're good to go!")

    else:
        print("\nNo null values in. Good to go, no need for cleaning :)")

    return dataset

In [3]:
#Connecting to pgAdmin

data_path = "."

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        HOST       = db_conn_dict['host']
        DB_USER    = db_conn_dict['user']
        DB_PW      = db_conn_dict['password']
        DEFAULT_DB = db_conn_dict['user']

        try:
            db = create_engine('postgres+psycopg2://'+DB_USER+':'+DB_PW+'@'+HOST+'/'+DEFAULT_DB, echo=False)
            conn = db.connect()
            print('connected')
        except Exception as e:
            print("unable to connect to the database")
            print(e)
            
        return db,conn

credfilepath = os.path.join(data_path, "data2x01_db.json")
db, conn = pgconnect(credfilepath)


connected


## Cleaning StatisticalAreas.csv and Uploading it

In [4]:
rawData = pd.read_csv("StatisticalAreas.csv")

#Check if contains NULL
cleanData = checkNull(rawData)

#Cleaning data so that only numeric values are included in area_id
if cleanData['area_id'].dtypes != int:
    print("\nUpdated area_id data type from " + str(cleanData['area_id'].dtypes) + " to integer & removed data with wrong type")
    cleanData = cleanData[cleanData.area_id.str.isnumeric()]
    cleanData['area_id'].astype(int)

#Cleaning data so that only numeric values are included in parent_area_id
if cleanData['parent_area_id'].dtypes != int:
    print("\nUpdated area_id data type from " + str(cleanData['parent_area_id'].dtypes) + " to integer & removed data with wrong type")
    cleanData = cleanData[cleanData.parent_area_id.str.isnumeric()]
    cleanData['parent_area_id'].astype(int)

statisticalAreas_clean = cleanData

#Uploading StatisticalAreas.csv to pgAdmin

conn.execute("DROP TABLE IF EXISTS statisticalareas")

statistical_areas = """CREATE TABLE IF NOT EXISTS statisticalareas (
                         area_id   Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         parent_area_id Integer
                   )"""
conn.execute(statistical_areas)
print("\nSuccessfully, created table for statistical areas...")

table_name = "statisticalareas"
statisticalAreas_clean.to_sql(table_name, con=conn, if_exists='replace',index=False)
print ("\nUploaded clean statistical areas data to pgAdmin")
res = pd.read_sql_query("SELECT * FROM statisticalareas",conn)
res


No null values in. Good to go, no need for cleaning :)

Successfully, created table for statistical areas...

Uploaded clean statistical areas data to pgAdmin


,area_id,area_name,parent_area_id
0,1,New South Wales,0
1,10,Greater Sydney,1
2,11,Rest of NSW,1
3,2,Victoria,0
4,20,Greater Melbourne,2
...,...,...,...
429,106,Hunter Valley exc Newcastle,11
430,11102,Lake Macquarie - West,111
431,111,Newcastle and Lake Macquarie,11
432,11402,Southern Highlands,114


## Cleaning and Uploading Neighbourhoods.csv

In [5]:
rawData = pd.read_csv("Neighbourhoods.csv")
#Check if there's null values

noNull_data = checkNull(rawData)
neighbourhoods_clean = noNull_data.copy()
numericData = ['area_id', 'land_area', 'population', 'number_of_dwellings', 'number_of_dwellings', 
               'number_of_businesses', 'median_annual_household_income', 'avg_monthly_rent']

for col in noNull_data.columns:
    
    if col in numericData:        
        if noNull_data[col].dtypes != int and noNull_data[col].dtypes != float:
            neighbourhoods_clean[col] = noNull_data[col].str.replace(',', '')
            neighbourhoods_clean[col] = pd.to_numeric(neighbourhoods_clean[col])   
            
print ("\nCommas in numeric data removed and all numbers as type string converted to numeric values!")

#Uploading Neighbourhoods.csv to pgAdmin

conn.execute("DROP TABLE IF EXISTS neighbourhoods")

neighbourhoods = """CREATE TABLE IF NOT EXISTS neighbourhoods (
                         area_id   Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         land_area Integer,
                         population Integer,
                         number_of_dwellings Integer,
                         number_of_businesses Integer,
                         median_annual_household_income Integer,
                         avg_monthly_rent Integer
                   )"""
conn.execute(neighbourhoods)
print("\nSuccessfully created neighbourhoods table.")

#Testing if table has been created
table_name = "neighbourhoods"
neighbourhoods_clean.to_sql(table_name, con=conn, if_exists='replace',index=False)

res = pd.read_sql_query('SELECT * FROM neighbourhoods', conn)
res


THIS DATASET HAS SOME NA VALUES.
Null value found. Coverted null values to ...
THIS DATASET HAS SOME NA VALUES.
Null value found. Coverted null values to ...
THIS DATASET HAS SOME NA VALUES.
Null value found. Coverted null values to ...
THIS DATASET HAS SOME NA VALUES.
Null value found. Coverted null values to ...

Null values have been removed. You're good to go!

Commas in numeric data removed and all numbers as type string converted to numeric values!

Successfully created neighbourhoods table.


,area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,median_annual_household_income,avg_monthly_rent
0,102011028,Avoca Beach - Copacabana,643.8000,7590,2325,738.0,46996.0,1906.0
1,102011029,Box Head - MacMasters Beach,3208.6000,10986,3847,907.0,42621.0,1682.0
2,102011030,Calga - Kulnura,76795.1000,4841,1575,1102.0,42105.0,1182.0
3,102011031,Erina - Green Point,3379.3000,14237,4450,1666.0,43481.0,1595.0
4,102011032,Gosford - Springfield,1691.2000,19385,6373,2126.0,45972.0,1382.0
...,...,...,...,...,...,...,...,...
304,106011109,Cessnock Region,1570.4341,7931,3281,673.0,73164.0,1080.0
305,106011113,Singleton Region,4067.2349,4919,2055,698.0,87984.0,1000.0
306,111021218,Morisset - Cooranbong,330.5208,14959,6298,1154.0,58084.0,1260.0
307,114021285,Hill Top - Colo Vale,174.3752,6025,2249,400.0,81120.0,1512.0


## Cleaning and Uploading BusinessStats.csv


In [ ]:
rawData = pd.read_csv("BusinessStats.csv")

#Remove null values
noNull_data = checkNull(rawData)
businesStats_clean = noNull_data.copy()

numericData = ['area_id', 'number_of_businesses', 'accommodation_and_food_services', 'retail_trade', 'agriculture_forestry_and_fishing', 
               'health_care_and_social_assistance', 'public_administration_and_safety', 'transport_postal_and_warehousing']

for col in noNull_data.columns:
    
    if col in numericData:        
        if noNull_data[col].dtypes != int and noNull_data[col].dtypes != float:
            businesStats_clean[col] = noNull_data[col].str.replace(',', '')
            businesStats_clean[col] = pd.to_numeric(businesStats_clean[col])   

            
print ("\nCommas in numeric data removed and all numbers as type string converted to numeric values!")

conn.execute("DROP TABLE IF EXISTS businessstats")

business_stats = """CREATE TABLE IF NOT EXISTS businessstats (
                         area_id   Integer PRIMARY KEY,
                         area_name VARCHAR(20),
                         number_of_businesses Integer,
                         accomodation_and_food_services Integer,
                         retail_trade Integer,
                         agriculture_forestry_and_fishing Integer,
                         health_care_and_social_assistance Integer,
                         public_administration_and_safety Integer,
                         transport_postal_and_warehousing Integer
                   )"""
conn.execute(business_stats)
print("Created table for business stats")

table_name = "businessstats"
businesStats_clean.to_sql(table_name, con=conn, if_exists='replace',index=False)

#Testing if table has been created
print ("Business Stats table created")
res = pd.read_sql_query('SELECT * FROM businessstats', conn)
res


No null values in. Good to go, no need for cleaning :)

Commas in numeric data removed and all numbers as type string converted to numeric values!
Created table for business stats


## Cleaning and Uploading RFSNSW_BFPL

In [ ]:
abs_pathname = os.path.abspath("RFSNSW_BFPL.shp")
rfs_shapefile = gpd.read_file(abs_pathname)
print(rfs_shapefile)
rfs_shapefile = checkNull(rfs_shapefile)


RFSNSW_BFPL = '''CREATE TABLE rfsnsw_bfpl (
                     "CATEGORY" INTEGER, 
                     "SHAPE_LENG" FLOAT, 
                     "SHAPE_AREA" FLOAT, 
                     location GEOMETRY(POINT,4326))''' 

conn.execute("DROP TABLE IF EXISTS rfsnsw_bfpl")
conn.execute(RFSNSW_BFPL)

rfs_tablename = "rfsnsw_bfpl"
rfs_shapefile.to_postgis(rfs_tablename, conn, if_exists='replace')

res = pd.read_sql_query('SELECT * FROM rfsnsw_bfpl', conn)
res

## Cleaning and Uploading SA2_2016_AUST

In [ ]:
abs_pathname = os.path.abspath("SA2_2016_AUST.shp")
sa_2_shapefile = gpd.read_file(abs_pathname)
print(sa_2_shapefile)
sa_2_shapefile = checkNull(sa_2_shapefile)
print(sa_2_shapefile.dtypes)


In [ ]:
SA2_2016 = '''CREATE TABLE sa2_2016 (
                     "SA2_MAIN16" NUMERIC, 
                     "SA2_5DIG16" NUMERIC, 
                     "SA2_NAME16" VARCHAR(50), 
                     "SA3_CODE16" NUMERIC, 
                     "SA3_NAME16" VARCHAR(50), 
                     "SA4_CODE16" NUMERIC,
                     "SA4_NAME16" VARCHAR(50), 
                     "GCC_CODE16" VARCHAR(50), 
                     "GCC_NAME16" VARCHAR(50),
                     "STE_CODE16" NUMERIC, 
                     "STE_NAME16" VARCHAR(50), 
                     "AREASQKM16" FLOAT,
                     location GEOMETRY(MULTIPOLYGON,4326))''' 

conn.execute("DROP TABLE IF EXISTS sa2_2016")
conn.execute(SA2_2016)

sa2_tablename = "sa2_2016"
sa_2_shapefile.to_postgis(sa2_tablename, conn, if_exists='replace')

res = pd.read_sql_query('SELECT * FROM sa2_2016', conn)
res

## Cleaning and Uploading Additional Data Set

In [ ]:
#Reading the json file
import json
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame

with open('RFSStation_EPSG4326.json') as f:
  fire_stations = json.load(f)
f.close()


i = 0
data_length = len(fire_stations['RFSStation']['features'])
null_found = False

while i < data_length:
    if len(fire_stations['RFSStation']['features'][i]['geometry']['coordinates']) < 2:
        print("Null coordinates found")
        null_found = True
        
    i += 1

if not null_found:
    print("No Null Coordinates in this file. You're good to go!")
    
fireStation_df = gpd.GeoDataFrame.from_features(fire_stations['RFSStation'])
fireStation_df.plot()

#Creating table for firestations
conn.execute("DROP TABLE IF EXISTS rfsfirestations")

RFS_FireStations = """CREATE TABLE IF NOT EXISTS rfsfirestations (
                        coordinates GEOMETRY PRIMARY KEY,
                        stationid INTEGER, 
                        station_name VARCHAR(20)
                   )"""

try:
     conn.execute(RFS_FireStations)
     print("Successfully created table for fire_stations")

except Exception as e:
     print("Table not created.\n")
     print(e)
    
fireStation_df.to_postgis('rfsfirestations', conn, if_exists='replace')
res = pd.read_sql_query('SELECT * FROM rfsfirestations', conn)
res


# Joining the Table

We are going to join the three CSV files.

In [ ]:
one = pd.read_sql_query("SELECT * FROM neighbourhoods",conn)
one

## Population Density

In [ ]:
two = pd.read_sql_query("SELECT population, land_area, population/land_area AS population_density FROM neighbourhoods",conn)
two

In [ ]:
three = pd.read_sql_query("SELECT number_of_dwellings, land_area FROM neighbourhoods",conn)
three

In [ ]:
four = pd.read_sql_query("SELECT number_of_dwellings, land_area, number_of_dwellings/land_area AS dwelling_density FROM neighbourhoods",conn)
four

In [ ]:
five = pd.read_sql_query("SELECT * FROM businessstats",conn)
five

# # Three Way Join

In [ ]:
six = pd.read_sql_query("SELECT * FROM businessstats B, neighbourhoods N, statisticalareas S WHERE B.area_id = S.area_id AND S.area_id = N.area_id",conn)
six

In [ ]:
seven = pd.read_sql_query("""SELECT N.population/N.land_area AS population_density,
                                    N.number_of_dwellings/N.land_area AS dwelling_density,
                                    B.number_of_businesses/N.land_area AS business_density,
                                    B.health_care_and_social_assistance/N.land_area AS assistive_service_density
                            FROM businessstats B, neighbourhoods N, statisticalareas S 
                            WHERE B.area_id = S.area_id 
                            AND S.area_id = N.area_id
                           
                          """
                          ,conn)
seven

Let's refer to the shape file now since we're nearly there.

In [ ]:
rfs_shapefile = rfs_shapefile.set_crs(epsg=4326,allow_override=True )
sa_2_shapefile = sa_2_shapefile.set_crs(epsg=4326,allow_override=True )

rfs_sa2 = gpd.sjoin(sa_2_shapefile, rfs_shapefile, op='contains')
rfs_sa2_frame = rfs_sa2['SA2_MAIN16'].to_frame()

In [ ]:
fireStation_df = fireStation_df.set_crs(epsg=4326,allow_override=True )

#Searching for points that is within the land specified
fireStation_sa2 = gpd.sjoin(fireStation_df, sa_2_shapefile, op='within')
fireStation_sa2

In [ ]:
fireStation_count = fireStation_sa2.groupby(['SA2_MAIN16']).count()
fireStation_count = fireStation_count['geometry'].to_frame()
fireStation_count['SA2_MAIN16_fireS'] = fireStation_count.index
fireStation_count = fireStation_count.rename(columns={'geometry': 'station_count'})


firestations_count_val = """CREATE TABLE IF NOT EXISTS firestations (
                        SA2_MAIN16 Integer PRIMARY KEY,
                        station_count Integer
                   )"""

conn.execute(firestations_count_val)
fireStation_count.to_sql('firestations', con=conn, if_exists='replace',index=False)
res = pd.read_sql_query('SELECT * FROM firestations', conn)
res

In [ ]:
rfs_sa2_frame = rfs_sa2.merge(fireStation_count, on="SA2_MAIN16")
rfs_sa2_frame